# Outline

# I. Introduction

# II. Preprocessing

In [ ]:
%%bash

# Following script saved as preprocessing.sh
# Command: sbatch preprocessing.sh experiment keyfile

###################################################################################################################
#! /bin/bash

#SBATCH --job-name wil_works
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=4
#SBATCH --partition=ccr
#SBATCH --time=24:00:00
#SBATCH --gres=lscratch:500 
#SBATCH --mem=50g


experiment=$1
keyfile=$2
reference=$3


# Extract tarball
tar -xvf ${experiment}.tar


# Base Calling
module load bcl2fastq/2.20.0 || exit 1
bcl2fastq --runfolder-dir ${experiment} -o ${experiment}_fastq_dir -r 4 -w 4 -p 14 --barcode-mismatches 0

# Concatenate all reads in one FASTQ file
zcat ${experiment}_fastq_dir/*.fastq.gz > ${experiment}.fastq

# Demultiplex FASTQ file
perl /data/BatistaLab_NGS/Scripts/icSHAPE-master/scripts/splitFastq.pl -U ${experiment}.fastq -b 6:6 -l CGTGAT:1::ACATCG:2::GCCTAA:3::TGGTCA:4::CACTGT:5::ATTGGC:6::GATCTG:7::TCAAGT:8::CTGATC:9::AAGCTA:10::GTAGCC:11::TACAAG:12::TTGACT:13::GGAACT:14::TGACAT:15::GGACGG:16::GCGGAC:17::TTTCAC:18::GGCCAC:19::CGAAAC:20::CGTACG:21::CCACTC:22::ATCAGT:23::AGGAAT:24 -d ${experiment}.splitFastq

# Compress FASTQ file
gzip -f ${experiment}.fastq

# Remove PCR duplicates
cd ${experiment}.splitFastq/
for file in *fastq; do perl /data/BatistaLab_NGS/Scripts/icSHAPE-master/scripts/readCollapse.pl -U ${file} ; done 

# Remove adapters
for file in *collapsed.fastq; do perl /data/BatistaLab_NGS/Scripts/icSHAPE-master/scripts/trimming.pl -U ${file} -o ${file}.trimm.fastq -l 17 -t 0 -c phred33 -a /data/BatistaLab_NGS/Scripts/s4U_pipeline/TruSeq2-PE.fa -m 20; done 
cd ..

# Rename FASTQ files according to KeyFile
while IFS=$'\t' read -r -a myArray
do
 cp  ${experiment}.splitFastq/${myArray[0]}.collapsed.fastq.trimm.fastq ${experiment}.splitFastq/${myArray[1]}.fastq
done < ${keyfile}

rm ${experiment}.splitFastq/*collapsed.fastq.trimm.fastq

# Prepare reference genome and  Align FASTQ files
module load bowtie

if ${reference} == "Ecoli"
then
  bowtie2-build /data/BatistaLab_NGS/Scripts/s4U_pipeline/transcriptome/transcriptome_uniquetRNA.fa Ecoli_transcriptome
  for file in ${experiment}.splitFastq/*rep*.fastq ; do bowtie2 -p 16 -x Ecoli_transcriptome -U ${file} -S ${file}.bam ; done
fi

if ${reference} == "Paeruginosa"
then
  bowtie2-build /data/BatistaLab_NGS/Scripts/s4U_pipeline/transcriptome/Pseudomonas_aeruginosa_transcriptome_tRNAunique.fa PA_transcriptome
  for file in ${experiment}.splitFastq/*rep*.fastq ; do bowtie2 -p 16 -x PA_transcriptome -U ${file} -S ${file}.bam ; done
fi


module load samtools

# Keep mapped reads
for file in ${experiment}.splitFastq/*fastq.bam ; do samtools view -F 4 -b -o ${file}.mapped.bam ${file} ; done

# Sort reads 
for file in ${experiment}.splitFastq/*.mapped.bam ; do samtools sort -o ${file}.sorted.bam ${file} ; done

# Create index
for file in ${experiment}.splitFastq/*.sorted.bam ; do samtools index ${file} ; done

# ??
    for file in ${experiment}.splitFastq/*fastq.bam.mapped.bam; do samtools view ${file} |  grep -E "@|NM:" |  grep -v "XS:" | cut -f 3|  sort | uniq -c | awk '{printf("%s\t%s\n", $2, $1)}' > ${file}.count ; done

# Rename COUNT files according to KeyFile
while IFS=$'\t' read -r -a myArray
do
 cp ${experiment}.splitFastq/${myArray[1]}.fastq.bam.mapped.bam.count ${experiment}.splitFastq/${myArray[1]}.count
done < ${keyfile}

####################################################################################################################


# III. Misincorporation HeatMaps

In [ ]:
#parse_pileups.py

import sys
import re

infile = open(sys.argv[1], 'rt')
infile.readline()
outfile = open(sys.argv[2], 'w+')

# List tRNAs ID from reference
tRNAs = ['EBT00001514652','EBT00001514586','EBT00001514664','EBT00001514749','EBT00001514687','EBT00001514602',\
         'EBT00001514635','EBT00001514579','EBT00001514747','EBT00001514671','EBT00001514668','EBT00001514736',\
         'EBT00001514738','EBT00001514720','EBT00001514654','EBT00001514578','EBT00001514721','EBT00001514692',\
         'EBT00001514633','EBT00001514638','EBT00001514676','EBT00001514740','EBT00001514753','EBT00001514655',\
         'EBT00001514732','EBT00001514606','EBT00001514688','EBT00001514601','EBT00001514622','EBT00001514729',\
         'EBT00001514735','EBT00001514716','EBT00001514697','EBT00001514650','EBT00001514699','EBT00001514691',\
         'EBT00001514582','EBT00001514702','EBT00001514593','EBT00001514626','EBT00001514730','EBT00001514592',\
         'EBT00001514696','EBT00001514711','EBT00001514647','EBT00001514636','EBT00001514709','EBT00001514679',\
         'EBT00001514624','EBT00001514739','EBT00001514751','EBT00001514722','EBT00001514600','EBT00001514728',\
         'EBT00001514597','EBT00001514667','EBT00001514651','EBT00001514715','EBT00001514737','EBT00001514613',\
         'EBT00001514723','EBT00001514693','EBT00001514637','EBT00001514632','EBT00001514733','EBT00001514743',\
         'EBT00001514689','EBT00001514584','EBT00001514690','EBT00001514678','EBT00001514596','EBT00001514707',\
         'EBT00001514627','EBT00001514591','EBT00001514734','EBT00001514641','EBT00001514612','EBT00001514598',\
         'EBT00001514684','EBT00001514719','EBT00001514665','EBT00001514621','EBT00001514583','EBT00001514604',\
         'EBT00001514683','EBT00001514642', # E coli

         'EBT00051365344','EBT00051365099','EBT00051370091','EBT00051369932','EBT00051369986','EBT00051367609',\
         'EBT00051369609','EBT00051366328','EBT00051367914','EBT00051366603','EBT00051366399','EBT00051365908',\
         'EBT00051369492','EBT00051366465','EBT00051366923','EBT00051368550','EBT00051368683','EBT00051365503',\
         'EBT00051369841','EBT00051365018','EBT00051367353','EBT00051365422','EBT00051365808','EBT00051368205',\
         'EBT00051369258','EBT00051365670','EBT00051364934','EBT00051365187','EBT00051369657','EBT00051366843',\
         'EBT00051366536','EBT00051368279','EBT00051367683','EBT00051368081','EBT00051367446','EBT00051370188',\
         'EBT00051369313','EBT00051369436','EBT00051370040','EBT00051370146','EBT00051367983','EBT00051367160',\
         'EBT00051366161'] # P a

# Extract relevant information from pileup files for tRNA only
for line in infile:
    if line[0] != '#':
        array = line.strip().split('\t')
        ID, position, ref, alt, INFO = array[0], array[1], array[3], array[4], array[7]

        if ID in tRNAs:
            alt = alt.split(',')
            x = re.search("DP=([0-9]+);AD=([0-9,\,]+)", INFO)
            DP, AD = x.group(1), x.group(2)
            AD = AD.split(',')

            outfile.write(ID+'\t'+position+'\t'+DP+'\t'+ref+'\t'+str(float(AD[0])/float(DP))+'\t')
            
            for i in range(0, len(alt)):
                outfile.write(alt[i]+'\t'+str(float(AD[i+1])/float(DP))+'\t')

            outfile.write('\n')
            
            
            

In [ ]:
%%bash

# Following script saved as pileup_analysis.sh
# Command: sbatch pileup_analysis.sh experiment

###################################################################################################################
#! /bin/bash

#SBATCH --job-name wil_works
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=4
#SBATCH --partition=ccr
#SBATCH --time=24:00:00
#SBATCH --gres=lscratch:500 
#SBATCH --mem=50g

experiment=$1
reference=$2

# Make pileups
module load samtools
#for file in ${experiment}.splitFastq/*bam.sorted.bam ; do bcftools mpileup -d 1000000000  ${file} -f transcriptome_uniquetRNA.fa -a INFO/AD > ${file}.pileup; done
for file in ${experiment}.splitFastq/*bam.sorted.bam ; do bcftools mpileup -d 1000000000  ${file} -f Pseudomonas_aeruginosa_transcriptome_tRNAunique.fa -a INFO/AD > ${file}.pileup; done

if ${reference} == "Ecoli"
then
  for file in ${experiment}.splitFastq/*bam.sorted.bam ; do bcftools mpileup -d 1000000000  ${file} -f transcriptome_uniquetRNA.fa -a INFO/AD > ${file}.pileup; done
fi

if ${reference} == "Paeruginosa"
then
  for file in ${experiment}.splitFastq/*bam.sorted.bam ; do bcftools mpileup -d 1000000000  ${file} -f Pseudomonas_aeruginosa_transcriptome_tRNAunique.fa -a INFO/AD > ${file}.pileup; done
fi

# Keep only tRNA
for file in ${experiment}.splitFastq/*.pileup ; do python3 parse_pileups.py ${file} ${file}.out ; done

# Rename files
for file in ${experiment}.splitFastq/*.pileup.out ; do mv "${file}" "`echo ${file} | sed 's/.fastq.bam.mapped.bam.sorted.bam//'`"; done

###################################################################################################################




In [ ]:
#summarize_pileups.py
#https://www.geeksforgeeks.org/how-to-read-multiple-text-files-from-folder-in-python/

import os
import sys
import pandas as pd

# Folder Path 
path = sys.argv[1]

# Change the directory 
os.chdir(path) 

all_rows = {}

# Read text files
def read_text_file(file_path): 
    with open(file_path, 'r') as f: 

        
        rows = {}
        
        for line in f:
            
            array = line.strip().split('\t')
            ID = array[0]+'|'+array[1]
            
            DP = array[2]

            variants = ''
            
            for field in array[3:]:
                if field != '<*>':
                    variants = variants+field+','
                else:
                    break

            if ID not in rows:
                rows[ID] = ''

                
            rows[ID] = rows[ID] + 'DP='+DP+','+ variants

            
    return(rows)

                

            
# iterate through all files
for file in os.listdir(): 
    # Check whether file is in text format or not 
    if file.endswith("pileup.out"):
    
        column = file.replace(".pileup.out", "")

        file_path = f"{path}{file}"
  
        # call read text file function 
        all_rows[column] =  read_text_file(file_path)

        
final_rows = {}

columns = []
for column in all_rows:
    columns.append(column)
    
    for row in all_rows[column]:
        if row not in final_rows:
            final_rows[row] = []
            
        final_rows[row].append(all_rows[column][row])
        
df = pd.DataFrame.from_dict(final_rows, orient='index', columns=columns)
df.to_csv('pileup_summary.csv')



```

mkdir summarize_pileups/
cp ${experiment}.splitFastq/*.pileup.out summarize_pileups/
cd summarize_pileups/
python3 summarize_pileups.py ./



```


In [ ]:
# misincorporation_heatmap_Ecoli.py

import pandas as pd
import numpy as np
import sys

translate = {'EBT00001514652' : 'alaT', 'EBT00001514586' : 'alaU', 'EBT00001514664' : 'alaV',\
             'EBT00001514749' : 'alaW', 'EBT00001514687' : 'alaX', 'EBT00001514602' : 'argQ',\
             'EBT00001514635' : 'argU', 'EBT00001514579' : 'argV', 'EBT00001514747' : 'argW',\
             'EBT00001514671' : 'argX', 'EBT00001514668' : 'argY', 'EBT00001514736' : 'argZ',\
             'EBT00001514738' : 'asnT', 'EBT00001514720' : 'asnU', 'EBT00001514654' : 'asnV',\
             'EBT00001514578' : 'asnW', 'EBT00001514721' : 'aspT', 'EBT00001514692' : 'aspU',\
             'EBT00001514633' : 'aspV', 'EBT00001514638' : 'cysT', 'EBT00001514676' : 'glnU',\
             'EBT00001514740' : 'glnV', 'EBT00001514753' : 'glnW', 'EBT00001514655' : 'glnX',\
             'EBT00001514732' : 'gltT', 'EBT00001514606' : 'gltU', 'EBT00001514688' : 'gltV',\
             'EBT00001514601' : 'gltW', 'EBT00001514622' : 'glyT', 'EBT00001514729' : 'glyU',\
             'EBT00001514735' : 'glyV', 'EBT00001514716' : 'glyW', 'EBT00001514697' : 'glyX',\
             'EBT00001514650' : 'glyY', 'EBT00001514699' : 'hisR', 'EBT00001514691' : 'ileT',\
             'EBT00001514582' : 'ileU', 'EBT00001514702' : 'ileV', 'EBT00001514593' : 'ileX',\
             'EBT00001514626' : 'ileY', 'EBT00001514730' : 'leuP', 'EBT00001514592' : 'leuQ',\
             'EBT00001514696' : 'leuT', 'EBT00001514711' : 'leuU', 'EBT00001514647' : 'leuV',\
             'EBT00001514636' : 'leuW', 'EBT00001514709' : 'leuX', 'EBT00001514679' : 'leuZ',\
             'EBT00001514624' : 'lysQ', 'EBT00001514739' : 'lysT', 'EBT00001514751' : 'lysV',\
             'EBT00001514722' : 'lysW', 'EBT00001514600' : 'lysY', 'EBT00001514728' : 'lysZ',\
             'EBT00001514597' : 'metT', 'EBT00001514667' : 'metU', 'EBT00001514651' : 'metV',\
             'EBT00001514715' : 'metW', 'EBT00001514737' : 'metY', 'EBT00001514613' : 'metZ',\
             'EBT00001514723' : 'pheU', 'EBT00001514693' : 'pheV', 'EBT00001514637' : 'proK',\
             'EBT00001514632' : 'proL', 'EBT00001514733' : 'proM', 'EBT00001514743' : 'selC',\
             'EBT00001514689' : 'serT', 'EBT00001514584' : 'serU', 'EBT00001514690' : 'serV',\
             'EBT00001514678' : 'serW', 'EBT00001514596' : 'serX', 'EBT00001514707' : 'thrT',\
             'EBT00001514627' : 'thrU', 'EBT00001514591' : 'thrV', 'EBT00001514734' : 'thrW',\
             'EBT00001514641' : 'trpT', 'EBT00001514612' : 'tyrT', 'EBT00001514598' : 'tyrU',\
             'EBT00001514684' : 'tyrV', 'EBT00001514719' : 'valT', 'EBT00001514665' : 'valU',\
             'EBT00001514621' : 'valV', 'EBT00001514583' : 'valW', 'EBT00001514604' : 'valX',\
             'EBT00001514683' : 'valY', 'EBT00001514642' : 'valZ'}


infile = pd.read_csv('pileup_summary.csv')


ID_POS = infile.iloc[:, 0]

experiment = sys.argv[1]

Rep1 = infile[experiment+'_rep1']
Rep2 = infile[experiment+'_rep2']
Rep3 = infile[experiment+'_rep3']

dic_depths = {}
dic = {}

for i in range(0, len(ID_POS)):
    
    ID, POS = ID_POS[i].split('|')
    
    DP_depths = []
    DP_refs = []
    
    for Rep in [Rep1, Rep2, Rep3]:
        
        if 'nan' in str(Rep[i]):
            
            DP_depths.append(np.nan)
            DP_refs.append(np.nan)
                
        else:
            DP = Rep[i].split(',')[0].split('=')[1]
            
            DP_ref = Rep[i].split(',')[2]

            if int(DP) > 1:
                DP_refs.append(float(DP_ref))
                DP_depths.append(int(DP))
            else:
                DP_refs.append(np.nan)  
                DP_depths.append(np.nan)
    
    DP_ref_mean = np.nanmean(DP_refs)
    
    ID = translate[ID]
    
    if ID not in dic:
        dic_depths[ID] = []
        dic[ID] = []
        
    dic_depths[ID].append(DP_depths)
    dic[ID].append(DP_ref_mean)
    

df = pd.DataFrame.from_dict(dic, orient='index')
df.to_csv(experiment+'_HeatMap.csv')

df_depths = pd.DataFrame.from_dict(dic_depths, orient='index')
df_depths.to_csv(experiment+'_HeatMap_depths.csv')





```
for experiment in M63p06_captured M63p06_input M63p2_captured M63p2_input M63m06_captured M63m06_input LB06_captured LB06_input LB2_captured LB2_input Spent0perc_captured Spent0perc_input Spent25perc_captured Spent25perc_input Spent50perc_captured Spent50perc_input dThiI06_input dThiI2_input ; do python3 misincorporation_heatmap_Ecoli.py ${experiment} ; done

```


In [ ]:
# misincorporation_heatmap_PA.py

import pandas as pd
import numpy as np
import sys


infile = pd.read_csv('PA_pileup_summary.csv')


ID_POS = infile.iloc[:, 0]

experiment = sys.argv[1]

Rep1 = infile[experiment+'_rep1']
Rep2 = infile[experiment+'_rep2']
Rep3 = infile[experiment+'_rep3']

dic_depths = {}
dic = {}

for i in range(0, len(ID_POS)):
    
    ID, POS = ID_POS[i].split('|')
    
    DP_depths = []
    DP_refs = []
    
    for Rep in [Rep1, Rep2, Rep3]:
        
        if 'nan' in str(Rep[i]):
            
            DP_depths.append(np.nan)
            DP_refs.append(np.nan)
                
        else:
            DP = Rep[i].split(',')[0].split('=')[1]
            
            DP_ref = Rep[i].split(',')[2]

            if int(DP) > 1:
                DP_refs.append(float(DP_ref))
                DP_depths.append(int(DP))
            else:
                DP_refs.append(np.nan)  
                DP_depths.append(np.nan)
    
    DP_ref_mean = np.nanmean(DP_refs)
        
    if ID not in dic:
        dic_depths[ID] = []
        dic[ID] = []
        
    dic_depths[ID].append(DP_depths)
    dic[ID].append(DP_ref_mean)
    

df = pd.DataFrame.from_dict(dic, orient='index')
df.to_csv(experiment+'_HeatMap.csv')

df_depths = pd.DataFrame.from_dict(dic_depths, orient='index')
df_depths.to_csv(experiment+'_HeatMap_depths.csv')






```

for experiment in PA_capture PA_input ; do python3 misincorporation_heatmap_PA.py ${experiment} ; done

```



In [ ]:
%%R


##########################################################################################
# https://www.r-bloggers.com/2015/09/passing-arguments-to-an-r-script-from-command-lines/
# heatmap.R

library(ggplot2)
library(heatmaply)
library(plotly)
library(orca)


args = commandArgs(trailingOnly=TRUE)


# test if there is at least one argument: if not, return an error
if (length(args)==0) {
  stop("At least one argument must be supplied (input file).n", call.=FALSE)
}



data <- read.csv(args[1], header=TRUE)
data <- cbind(data[,1], data[,2:76])
colnames(data) <- c('ID', seq(1,length(data[1,])-1))

toplot <- data[-1]
rownames(toplot) <- data$ID
toplot <- toplot[order(rownames(toplot)), ]


p <- heatmaply(toplot, 
        dendrogram = "none",
        file = paste0(args[1],'.pdf'),
        #plot_method = "plotly",
        xlab = "", ylab = "", 
        main = args[1],
        #scale = "row",
        margins = c(60,100,40,20),
        grid_color = "white",
        grid_width = 0.00001,
        titleX = FALSE,
        hide_colorbar = TRUE,
        branches_lwd = 0.1,
        fontsize_row = 5, fontsize_col = 5,
        labRow = rownames(toplot),
        column_text_angle = 90,
        )
#############################################################################################


```
conda create -n heatmaps python=3.6
conda activate heatmaps
conda config --env --add channels bioconda
conda config --env --add channels conda-forge
conda config --env --set channel_priority strict
conda config --show-sources

conda install -c conda-forge r-base
conda install -c conda-forge r-ggplot2
conda install -c conda-forge r-heatmaply
conda install -c plotly plotly

R
install.packages("orca")

for experiment in M63p06_captured M63p06_input M63p2_captured M63p2_input M63m06_captured M63m06_input LB06_captured LB06_input LB2_captured LB2_input Spent0perc_captured Spent0perc_input Spent25perc_captured Spent25perc_input Spent50perc_captured Spent50perc_input dThiI06_input dThiI2_input ; do Rscript heatmap.R ${experiment}_HeatMap.csv ; done



```


Example of heatmap <img src="M63p06_captured_HeatMap.csv.pdf" width=1200 height=800 />


```
for experiment in PA_capture PA_input ; do Rscript heatmap.R ${experiment}_HeatMap.csv ; done

```

# IV. Termination HeatMaps

In [ ]:
%%bash

# Following script saved as termination_analysis.sh
# Command: sbatch termination_analysis.sh experiment

###################################################################################################################
#! /bin/bash

#SBATCH --job-name wil_works
#SBATCH --ntasks=4
#SBATCH --cpus-per-task=4
#SBATCH --partition=ccr
#SBATCH --time=24:00:00
#SBATCH --gres=lscratch:500 
#SBATCH --mem=50g

experiment=$1
reference=$2

module load bedtools

if ${reference} == "Ecoli"
then
  for file in ${experiment}.splitFastq/*bam.sorted.bam ; do bedtools genomecov -d -5 -ibam  ${file} -g transcriptome_uniquetRNA.fa > ${file}_5end.txt ; bedtools genomecov -d -3 -ibam ${file} -g transcriptome_uniquetRNA.fa > ${file}_3end.txt ; done
fi

if ${reference} == "Paeruginosa"
then
  for file in ${experiment}.splitFastq/*bam.sorted.bam ; do bedtools genomecov -d -5 -ibam  ${file} -g Pseudomonas_aeruginosa_transcriptome_tRNAunique.fa > ${file}_5end.txt ; bedtools genomecov -d -3 -ibam ${file} -g transcriptome_uniquetRNA.fa > ${file}_3end.txt ; done
fi


###################################################################################################################





In [ ]:
#summarize_terminations.py
#https://www.geeksforgeeks.org/how-to-read-multiple-text-files-from-folder-in-python/

import os
import sys
import pandas as pd


tRNAs = ['EBT00001514652','EBT00001514586','EBT00001514664','EBT00001514749','EBT00001514687','EBT00001514602',\
         'EBT00001514635','EBT00001514579','EBT00001514747','EBT00001514671','EBT00001514668','EBT00001514736',\
         'EBT00001514738','EBT00001514720','EBT00001514654','EBT00001514578','EBT00001514721','EBT00001514692',\
         'EBT00001514633','EBT00001514638','EBT00001514676','EBT00001514740','EBT00001514753','EBT00001514655',\
         'EBT00001514732','EBT00001514606','EBT00001514688','EBT00001514601','EBT00001514622','EBT00001514729',\
         'EBT00001514735','EBT00001514716','EBT00001514697','EBT00001514650','EBT00001514699','EBT00001514691',\
         'EBT00001514582','EBT00001514702','EBT00001514593','EBT00001514626','EBT00001514730','EBT00001514592',\
         'EBT00001514696','EBT00001514711','EBT00001514647','EBT00001514636','EBT00001514709','EBT00001514679',\
         'EBT00001514624','EBT00001514739','EBT00001514751','EBT00001514722','EBT00001514600','EBT00001514728',\
         'EBT00001514597','EBT00001514667','EBT00001514651','EBT00001514715','EBT00001514737','EBT00001514613',\
         'EBT00001514723','EBT00001514693','EBT00001514637','EBT00001514632','EBT00001514733','EBT00001514743',\
         'EBT00001514689','EBT00001514584','EBT00001514690','EBT00001514678','EBT00001514596','EBT00001514707',\
         'EBT00001514627','EBT00001514591','EBT00001514734','EBT00001514641','EBT00001514612','EBT00001514598',\
         'EBT00001514684','EBT00001514719','EBT00001514665','EBT00001514621','EBT00001514583','EBT00001514604',\
         'EBT00001514683','EBT00001514642', # E coli

         'EBT00051365344','EBT00051365099','EBT00051370091','EBT00051369932','EBT00051369986','EBT00051367609',\
         'EBT00051369609','EBT00051366328','EBT00051367914','EBT00051366603','EBT00051366399','EBT00051365908',\
         'EBT00051369492','EBT00051366465','EBT00051366923','EBT00051368550','EBT00051368683','EBT00051365503',\
         'EBT00051369841','EBT00051365018','EBT00051367353','EBT00051365422','EBT00051365808','EBT00051368205',\
         'EBT00051369258','EBT00051365670','EBT00051364934','EBT00051365187','EBT00051369657','EBT00051366843',\
         'EBT00051366536','EBT00051368279','EBT00051367683','EBT00051368081','EBT00051367446','EBT00051370188',\
         'EBT00051369313','EBT00051369436','EBT00051370040','EBT00051370146','EBT00051367983','EBT00051367160',\
         'EBT00051366161'] # P a



# Folder Path 
path = sys.argv[1]

# Change the directory 
os.chdir(path) 



all_rows = {}

# Read text File
def read_text_file(file_path): 
    with open(file_path, 'r') as f: 
        
        rows = {}
        
        for line in f:
            
            array = line.strip().split('\t')
            ID = array[0]+'|'+array[1]
            
            if array[0] in tRNAs:
                Term_count = array[2]
                
                if ID not in rows:
                    rows[ID] = ''

                rows[ID] = Term_count

    return(rows)

                

            
# iterate through all file 
for file in os.listdir(): 
    # Check whether file is in text format or not 
    if file.endswith("bam_5end.txt"):
    
        column = file.replace(".fastq.bam.mapped.bam.sorted.bam_5end.txt", "")

        file_path = f"{path}{file}"
  
        # call read text file function 
        all_rows[column] =  read_text_file(file_path)

        
final_rows = {}

columns = []
for column in all_rows:
    columns.append(column)
    
    for row in all_rows[column]:
        if row not in final_rows:
            final_rows[row] = []
            
        final_rows[row].append(all_rows[column][row])
        
df = pd.DataFrame.from_dict(final_rows, orient='index', columns=columns)
df.to_csv('termination_summary.csv')



```
mkdir summarize_terminations/
cp ${experiment}.splitFastq/*bam_5end.txt summarize_terminations/
cd summarize_terminations/
python3 summarize_terminations.py ./




```


In [ ]:
# termination_heatmap_Ecoli.py


import pandas as pd
import numpy as np
import sys

translate = {'EBT00001514652' : 'alaT', 'EBT00001514586' : 'alaU', 'EBT00001514664' : 'alaV',\
             'EBT00001514749' : 'alaW', 'EBT00001514687' : 'alaX', 'EBT00001514602' : 'argQ',\
             'EBT00001514635' : 'argU', 'EBT00001514579' : 'argV', 'EBT00001514747' : 'argW',\
             'EBT00001514671' : 'argX', 'EBT00001514668' : 'argY', 'EBT00001514736' : 'argZ',\
             'EBT00001514738' : 'asnT', 'EBT00001514720' : 'asnU', 'EBT00001514654' : 'asnV',\
             'EBT00001514578' : 'asnW', 'EBT00001514721' : 'aspT', 'EBT00001514692' : 'aspU',\
             'EBT00001514633' : 'aspV', 'EBT00001514638' : 'cysT', 'EBT00001514676' : 'glnU',\
             'EBT00001514740' : 'glnV', 'EBT00001514753' : 'glnW', 'EBT00001514655' : 'glnX',\
             'EBT00001514732' : 'gltT', 'EBT00001514606' : 'gltU', 'EBT00001514688' : 'gltV',\
             'EBT00001514601' : 'gltW', 'EBT00001514622' : 'glyT', 'EBT00001514729' : 'glyU',\
             'EBT00001514735' : 'glyV', 'EBT00001514716' : 'glyW', 'EBT00001514697' : 'glyX',\
             'EBT00001514650' : 'glyY', 'EBT00001514699' : 'hisR', 'EBT00001514691' : 'ileT',\
             'EBT00001514582' : 'ileU', 'EBT00001514702' : 'ileV', 'EBT00001514593' : 'ileX',\
             'EBT00001514626' : 'ileY', 'EBT00001514730' : 'leuP', 'EBT00001514592' : 'leuQ',\
             'EBT00001514696' : 'leuT', 'EBT00001514711' : 'leuU', 'EBT00001514647' : 'leuV',\
             'EBT00001514636' : 'leuW', 'EBT00001514709' : 'leuX', 'EBT00001514679' : 'leuZ',\
             'EBT00001514624' : 'lysQ', 'EBT00001514739' : 'lysT', 'EBT00001514751' : 'lysV',\
             'EBT00001514722' : 'lysW', 'EBT00001514600' : 'lysY', 'EBT00001514728' : 'lysZ',\
             'EBT00001514597' : 'metT', 'EBT00001514667' : 'metU', 'EBT00001514651' : 'metV',\
             'EBT00001514715' : 'metW', 'EBT00001514737' : 'metY', 'EBT00001514613' : 'metZ',\
             'EBT00001514723' : 'pheU', 'EBT00001514693' : 'pheV', 'EBT00001514637' : 'proK',\
             'EBT00001514632' : 'proL', 'EBT00001514733' : 'proM', 'EBT00001514743' : 'selC',\
             'EBT00001514689' : 'serT', 'EBT00001514584' : 'serU', 'EBT00001514690' : 'serV',\
             'EBT00001514678' : 'serW', 'EBT00001514596' : 'serX', 'EBT00001514707' : 'thrT',\
             'EBT00001514627' : 'thrU', 'EBT00001514591' : 'thrV', 'EBT00001514734' : 'thrW',\
             'EBT00001514641' : 'trpT', 'EBT00001514612' : 'tyrT', 'EBT00001514598' : 'tyrU',\
             'EBT00001514684' : 'tyrV', 'EBT00001514719' : 'valT', 'EBT00001514665' : 'valU',\
             'EBT00001514621' : 'valV', 'EBT00001514583' : 'valW', 'EBT00001514604' : 'valX',\
             'EBT00001514683' : 'valY', 'EBT00001514642' : 'valZ'}


infile = pd.read_csv('termination_summary.csv')


ID_POS = infile.iloc[:, 0]

experiment = sys.argv[1]

Rep1 = infile[experiment+'_rep1']
Rep2 = infile[experiment+'_rep2']
Rep3 = infile[experiment+'_rep3']


dic = {}

for i in range(0, len(ID_POS)):
    
    ID, POS = ID_POS[i].split('|')
    
    Terminations = []

    
    for Rep in [Rep1, Rep2, Rep3]:
        
        if 'nan' in str(Rep[i]):
            
            Terminations.append(np.nan)
                
        else:
            Terminations.append(Rep[i])

    Terminations_mean = np.nanmean(Terminations)
    
    ID = translate[ID]
    
    if ID not in dic:
        dic[ID] = []
        
    dic[ID].append(Terminations_mean)
    
    
    
dic_normalized = {}

for ID in dic:
    dic_normalized[ID] = []
    
    for i in range(0, len(dic[ID])):

        dic_normalized[ID].append(1- float(dic[ID][i])/ sum(dic[ID][:i+1]))
    
    

df = pd.DataFrame.from_dict(dic_normalized, orient='index')


df.to_csv(experiment+'_termination_HeatMap.csv')



```
for experiment in M63p06_captured M63p06_input M63p2_captured M63p2_input M63m06_captured M63m06_input LB06_captured LB06_input LB2_captured LB2_input Spent0perc_captured Spent0perc_input Spent25perc_captured Spent25perc_input Spent50perc_captured Spent50perc_input dThiI06_input dThiI2_input ; do python3 termination_heatmap_Ecoli.py ${experiment} ; done

for experiment in M63p06_captured M63p06_input M63p2_captured M63p2_input M63m06_captured M63m06_input LB06_captured LB06_input LB2_captured LB2_input Spent0perc_captured Spent0perc_input Spent25perc_captured Spent25perc_input Spent50perc_captured Spent50perc_input dThiI06_input dThiI2_input ; do Rscript heatmap.R ${experiment}_termination_HeatMap.csv ; done

```


In [ ]:
# termination_heatmap_PA.py


import pandas as pd
import numpy as np
import sys



infile = pd.read_csv('PA_termination_summary.csv')


ID_POS = infile.iloc[:, 0]

experiment = sys.argv[1]

Rep1 = infile[experiment+'_rep1']
Rep2 = infile[experiment+'_rep2']
Rep3 = infile[experiment+'_rep3']


dic = {}

for i in range(0, len(ID_POS)):
    
    ID, POS = ID_POS[i].split('|')
    
    Terminations = []

    
    for Rep in [Rep1, Rep2, Rep3]:
        
        if 'nan' in str(Rep[i]):
            
            Terminations.append(np.nan)
                
        else:
            Terminations.append(Rep[i])

    Terminations_mean = np.nanmean(Terminations)
        
    if ID not in dic:
        dic[ID] = []
        
    dic[ID].append(Terminations_mean)
    
    
    
dic_normalized = {}

for ID in dic:
    dic_normalized[ID] = []
    
    for i in range(0, len(dic[ID])):

        dic_normalized[ID].append(1- float(dic[ID][i])/ sum(dic[ID][:i+1]))
    
    

df = pd.DataFrame.from_dict(dic_normalized, orient='index')


df.to_csv(experiment+'_termination_HeatMap.csv')




```

for experiment in PA_capture PA_input ; do python3 termination_heatmap_PA.py ${experiment} ; done

for experiment in PA_capture PA_input ; do Rscript heatmap.R ${experiment}_termination_HeatMap.csv ; done
```



# DeSeq2 Analysis

## Keep only tRNAs

In [ ]:
#parse_tRNA.py

import sys
import re

infile = open(sys.argv[1], 'rt')
outfile = open(sys.argv[2], 'w+')


tRNAs = ['EBT00001514652','EBT00001514586','EBT00001514664','EBT00001514749','EBT00001514687','EBT00001514602',\
         'EBT00001514635','EBT00001514579','EBT00001514747','EBT00001514671','EBT00001514668','EBT00001514736',\
         'EBT00001514738','EBT00001514720','EBT00001514654','EBT00001514578','EBT00001514721','EBT00001514692',\
         'EBT00001514633','EBT00001514638','EBT00001514676','EBT00001514740','EBT00001514753','EBT00001514655',\
         'EBT00001514732','EBT00001514606','EBT00001514688','EBT00001514601','EBT00001514622','EBT00001514729',\
         'EBT00001514735','EBT00001514716','EBT00001514697','EBT00001514650','EBT00001514699','EBT00001514691',\
         'EBT00001514582','EBT00001514702','EBT00001514593','EBT00001514626','EBT00001514730','EBT00001514592',\
         'EBT00001514696','EBT00001514711','EBT00001514647','EBT00001514636','EBT00001514709','EBT00001514679',\
         'EBT00001514624','EBT00001514739','EBT00001514751','EBT00001514722','EBT00001514600','EBT00001514728',\
         'EBT00001514597','EBT00001514667','EBT00001514651','EBT00001514715','EBT00001514737','EBT00001514613',\
         'EBT00001514723','EBT00001514693','EBT00001514637','EBT00001514632','EBT00001514733','EBT00001514743',\
         'EBT00001514689','EBT00001514584','EBT00001514690','EBT00001514678','EBT00001514596','EBT00001514707',\
         'EBT00001514627','EBT00001514591','EBT00001514734','EBT00001514641','EBT00001514612','EBT00001514598',\
         'EBT00001514684','EBT00001514719','EBT00001514665','EBT00001514621','EBT00001514583','EBT00001514604',\
         'EBT00001514683','EBT00001514642', # E coli

         'EBT00051365344','EBT00051365099','EBT00051370091','EBT00051369932','EBT00051369986','EBT00051367609',\
         'EBT00051369609','EBT00051366328','EBT00051367914','EBT00051366603','EBT00051366399','EBT00051365908',\
         'EBT00051369492','EBT00051366465','EBT00051366923','EBT00051368550','EBT00051368683','EBT00051365503',\
         'EBT00051369841','EBT00051365018','EBT00051367353','EBT00051365422','EBT00051365808','EBT00051368205',\
         'EBT00051369258','EBT00051365670','EBT00051364934','EBT00051365187','EBT00051369657','EBT00051366843',\
         'EBT00051366536','EBT00051368279','EBT00051367683','EBT00051368081','EBT00051367446','EBT00051370188',\
         'EBT00051369313','EBT00051369436','EBT00051370040','EBT00051370146','EBT00051367983','EBT00051367160',\
         'EBT00051366161'] # P a



for line in infile:
    if line[0] != '#':
        array = line.strip().split('\t')
        ID, count = array
        
        if ID in tRNAs:

            outfile.write(ID+'\t'+count+'\n')

            
            
            

```
for file in *count; do python parse_tRNA.py ${file} ${file}.tRNA ; done

```


## Single-Factor analysis

In [ ]:
%%R

#SingleFactorDeSeq2_tRNA.r

library(dplyr)

args = commandArgs(trailingOnly=TRUE)
path <- args[1]
current <- args[2]
other <- args[3]

################################################
# PART 1: GATHER ALL INPUTS INTO ONE DATAFRAME #
###############################################

# LIST ALL INPUT FILES
temp <- list.files(path = path, pattern="*.count.tRNA")

# START DATAFRAME WITH FIRST INPUT
df <- as.data.frame(read.csv(paste0(path,temp[1]), header=FALSE, sep='\t'))
cols <- c('ID',substr(temp[1], 1, nchar(temp[1])-11))
condition <- substr(temp[1], 1, nchar(temp[1])-16)

# FILL DATAFRAME WITH THE REMAINING INPUTS
for (file in temp[2:length(temp)]) {
    cols <- c(cols,substr(file, 1, nchar(file)-11))
    condition <- c(condition,substr(file, 1, nchar(file)-16))
    df2 <- as.data.frame(read.csv(paste0(path,file), header=FALSE, sep='\t'))
    df <- full_join(df, df2, by='V1')
}

condition <- factor(condition)
warnings()
df[is.na(df)] <- 0
colnames(df) <- cols
write.table(df,"s4U_counts_table.tRNA.txt", quote = FALSE, row.names = FALSE)


##################################
# PART 2: SINGLE FACTOR ANALYSIS #
##################################

# READ DATA INTO MATRIX
data <- read.table("s4U_counts_table.tRNA.txt", header=T, row.names=1)
countdata <- as.matrix(data)

# MAKE MATRIX A DESEQ2 OBJECT
library(DESeq2)
coldata <- data.frame(row.names=colnames(countdata), condition)
print(coldata)
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=coldata, design=~condition)
print(summary(dds))
dds

#filter the dataset - remove all zeros, influences padj
nrow(dds)
dds <- dds[ rowSums(counts(dds)) > 1, ]
nrow(dds)

# Run the DESeq pipeline
dds <- DESeq(dds)
res <- results(dds)
print(res)


conditions <- c(paste0(current,'_captured'), paste0(current,'_input'), paste0(other,'_captured'), paste0(other,'_input'))

for (current in conditions) { conditions <- conditions[conditions != current]
                             
        for (other in conditions) {
            
            print(current)
            print(other)
    
            current_res = results(dds, contrast=c("condition",current,other))
            current_res
            print(current_res)
            summary(current_res)
            current_ress_Orderedlfc <- current_res[order(current_res$log2FoldChange),]
            head(current_res_Orderedlfc)

            # Exploring and exporting results

            pdf(file = paste0(current,'_VS_',other,'.pdf'),   # The directory you want to save the file in
            width = 10, # The width of the plot in inches
            height = 10) # The height of the plot in inches

            plotMA(current_res, ylim=c(-10,10))
            dev.off()

            identify(current_res$baseMean, current_res$log2FoldChange, labels = row.names(current_res))
            write.csv(as.data.frame(current_res), file= paste0(current,'_VS_',other,'.tRNA.csv'))

            # Volcano plot
            pdf(file = paste0(current,'_VS_',other,'.volcano.tRNA.pdf'),   # The directory you want to save the file in
                width = 10, # The width of the plot in inches
                height = 10) # The height of the plot in inches
            par(mar=c(5,5,5,5), cex=1.0, cex.main=1.4, cex.axis=1.4, cex.lab=1.4)
            topT <- as.data.frame(current_res)
            #Adjusted P values (FDR Q values)
            with(topT, plot(log2FoldChange, -log10(padj), pch=20, main=paste0(current,' VS ',other,'.volcano'), cex=1.0, xlab=bquote(~Log[2]~fold~change), ylab=bquote(~-log[10]~Q~value)))
            with(subset(topT, padj<0.05 & abs(log2FoldChange)>1), points(log2FoldChange, -log10(padj), pch=20, col="red", cex=0.5))
            #Add lines for absolute FC>2 and P-value cut-off at FDR Q<0.05
            abline(v=0, col="black", lty=3, lwd=1.0)
            abline(v=-1, col="black", lty=4, lwd=2.0)
            abline(v=1, col="black", lty=4, lwd=2.0)
            abline(h=-log10(max(topT$pvalue[topT$padj<0.05], na.rm=TRUE)), col="black", lty=4, lwd=2.0)
            dev.off()
        }
    }
            




```

Rscript SingleFactorDeSeq2_tRNA.r LBvsM63analysis_tRNA/ LB06 LB2
Rscript SingleFactorDeSeq2_tRNA.r LBvsM63analysis_tRNA/ M63m06 M63p06
Rscript SingleFactorDeSeq2_tRNA.r LBvsM63analysis_tRNA/ M63p06 M63p2
Rscript SingleFactorDeSeq2_tRNA.r LBvsM63analysis_tRNA/ LB06 M63p06
Rscript SingleFactorDeSeq2_tRNA.r LBvsM63analysis_tRNA/ LB2 M63p2



Rscript SingleFactorDeSeq2_tRNA.r Spent_analysis_tRNA/ Spent0perc Spent25perc
Rscript SingleFactorDeSeq2_tRNA.r Spent_analysis_tRNA/ Spent0perc Spent50perc
Rscript SingleFactorDeSeq2_tRNA.r Spent_analysis_tRNA/ Spent25perc Spent50perc



```



In [ ]:
%%R

#SingleFactorDeSeq2_PA_tRNA.r

library(dplyr)

args = commandArgs(trailingOnly=TRUE)
path <- args[1]
current <- args[2]
other <- args[3]

################################################
# PART 1: GATHER ALL INPUTS INTO ONE DATAFRAME #
###############################################

# LIST ALL INPUT FILES
temp <- list.files(path = path, pattern="*.count.tRNA")

# START DATAFRAME WITH FIRST INPUT
df <- as.data.frame(read.csv(paste0(path,temp[1]), header=FALSE, sep='\t'))
cols <- c('ID',substr(temp[1], 1, nchar(temp[1])-11))
condition <- substr(temp[1], 1, nchar(temp[1])-16)

# FILL DATAFRAME WITH THE REMAINING INPUTS
for (file in temp[2:length(temp)]) {
    cols <- c(cols,substr(file, 1, nchar(file)-11))
    condition <- c(condition,substr(file, 1, nchar(file)-16))
    df2 <- as.data.frame(read.csv(paste0(path,file), header=FALSE, sep='\t'))
    df <- full_join(df, df2, by='V1')
}

condition <- c(rep("PA_input",3), rep("PA_capture",3))
condition <- factor(condition)
warnings()
df[is.na(df)] <- 0
colnames(df) <- cols
write.table(df,"s4U_counts_table.tRNA.txt", quote = FALSE, row.names = FALSE)


##################################
# PART 2: SINGLE FACTOR ANALYSIS #
##################################

# READ DATA INTO MATRIX
data <- read.table("s4U_counts_table.tRNA.txt", header=T, row.names=1)
countdata <- as.matrix(data)

# MAKE MATRIX A DESEQ2 OBJECT
library(DESeq2)
coldata <- data.frame(row.names=colnames(countdata), condition)
print(coldata)
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=coldata, design=~condition)
print(summary(dds))
dds

#filter the dataset - remove all zeros, influences padj
nrow(dds)
dds <- dds[ rowSums(counts(dds)) > 1, ]
nrow(dds)

# Run the DESeq pipeline
dds <- DESeq(dds)
res <- results(dds)
print(res)


conditions <- c(paste0(current,'_input'), paste0(other,'_captured'))

for (current in conditions) { conditions <- conditions[conditions != current]
                             
        for (other in conditions) {
            
            print(current)
            print(other)
    
            current_res = results(dds)
            current_res
            print(current_res)
            summary(current_res)
            current_ress_Orderedlfc <- current_res[order(current_res$log2FoldChange),]
            head(current_res_Orderedlfc)

            # Exploring and exporting results

            pdf(file = paste0(current,'_VS_',other,'.pdf'),   # The directory you want to save the file in
            width = 10, # The width of the plot in inches
            height = 10) # The height of the plot in inches

            plotMA(current_res, ylim=c(-10,10))
            dev.off()

            identify(current_res$baseMean, current_res$log2FoldChange, labels = row.names(current_res))
            write.csv(as.data.frame(current_res), file= paste0(current,'_VS_',other,'.tRNA.csv'))

            # Volcano plot
            pdf(file = paste0(current,'_VS_',other,'.volcano.tRNA.pdf'),   # The directory you want to save the file in
                width = 10, # The width of the plot in inches
                height = 10) # The height of the plot in inches
            par(mar=c(5,5,5,5), cex=1.0, cex.main=1.4, cex.axis=1.4, cex.lab=1.4)
            topT <- as.data.frame(current_res)
            #Adjusted P values (FDR Q values)
            with(topT, plot(log2FoldChange, -log10(padj), pch=20, main=paste0(current,' VS ',other,'.volcano'), cex=1.0, xlab=bquote(~Log[2]~fold~change), ylab=bquote(~-log[10]~Q~value)))
            with(subset(topT, padj<0.05 & abs(log2FoldChange)>1), points(log2FoldChange, -log10(padj), pch=20, col="red", cex=0.5))
            #Add lines for absolute FC>2 and P-value cut-off at FDR Q<0.05
            abline(v=0, col="black", lty=3, lwd=1.0)
            abline(v=-1, col="black", lty=4, lwd=2.0)
            abline(v=1, col="black", lty=4, lwd=2.0)
            abline(h=-log10(max(topT$pvalue[topT$padj<0.05], na.rm=TRUE)), col="black", lty=4, lwd=2.0)
            dev.off()
        }
    }
            





```
Rscript SingleFactorDeSeq2_tRNA.r PA_analysis/ PA PA
```


## Multi-Factor Analysis

In [ ]:
%%R

#MultiFactorDeSeq2_tRNA.r

library(dplyr)

args = commandArgs(trailingOnly=TRUE)
path <- args[1]
current <- args[2]
other <- args[3]

################################################
# PART 1: GATHER ALL INPUTS INTO ONE DATAFRAME #
###############################################

# LIST ALL INPUT FILES
pattern1 = paste0(current,"*.count.tRNA", collapse = "|")
pattern2 = paste0(other,"*.count.tRNA", collapse = "|")
temp <- list.files(path = path, pattern=glob2rx(pattern1))
temp <- c(temp, list.files(path = path, pattern=glob2rx(pattern2)))


# START DATAFRAME WITH FIRST INPUT
df <- as.data.frame(read.csv(paste0(path,temp[1]), header=FALSE, sep='\t'))
cols <- c('ID',substr(temp[1], 1, nchar(temp[1])-6))
condition <- substr(temp[1], 1, nchar(temp[1])-11)

# FILL DATAFRAME WITH THE REMAINING INPUTS
for (file in temp[2:length(temp)]) {
    cols <- c(cols,substr(file, 1, nchar(file)-6))
    condition <- c(condition,substr(file, 1, nchar(file)-11))
    df2 <- as.data.frame(read.csv(paste0(path,file), header=FALSE, sep='\t'))
    df <- full_join(df, df2, by='V1')
}

condition <- factor(condition)
warnings()
df[is.na(df)] <- 0
colnames(df) <- cols
write.table(df,"s4U_counts_table.tRNA.txt", quote = FALSE, row.names = FALSE)


##################################
# PART 2: MULTI FACTOR ANALYSIS #
##################################

# READ DATA INTO MATRIX
data <- read.table("s4U_counts_table.tRNA.txt", header=T, row.names=1)
countdata <- as.matrix(data)

# MAKE MATRIX A DESEQ2 OBJECT
library(DESeq2)
#coldata <- data.frame(row.names=colnames(countdata), condition)
#dds <- DESeqDataSetFromMatrix(countData=countdata, colData=coldata, design=~condition)
#dds

#filter the dataset - remove all zeros, influences padj
#nrow(dds)
#dds <- dds[ rowSums(counts(dds)) > 1, ]
#nrow(dds)

# Run the DESeq pipeline
#dds <- DESeq(dds)
#res <- results (dds)


condition <- colnames(countdata)

MF_condition <- c()
MF_assay <- c()
MF_replicate <- c()

for (row in condition) {

        MF_condition <- c(MF_condition, unlist(strsplit(as.character(row), '_'))[1])
        MF_assay <- c(MF_assay, unlist(strsplit(as.character(row), '_'))[2])
        MF_replicate <- c(MF_replicate, unlist(strsplit(as.character(row), '_'))[3])
        }

MF_coldata <- data.frame(row.names=colnames(countdata), MF_condition, MF_assay, MF_replicate)
#MF_coldata <- MF_coldata[15:21,]
MF_coldata <- MF_coldata[,1:2]

print(MF_coldata)

ddsMF <- DESeqDataSetFromMatrix(countData=countdata, colData=MF_coldata, design=~ MF_assay + MF_condition + MF_assay:MF_condition)


print(ddsMF)

nrow(ddsMF)
ddsMF <- ddsMF[ rowSums(counts(ddsMF)) > 1, ]
nrow(ddsMF)


# Diferential expression analysis

current <- args[2]
other <- args[3]

ddsMF <- DESeq(ddsMF, test="LRT", reduced= ~ MF_assay + MF_condition) #creates the analysis
resultsNames(ddsMF)


print('MILESTONE')

#normalized_counts <- counts(ddsMF, normalized=TRUE)
#write.table(normalized_counts, file="normalized_counts.ddMF.txt", sep=",", quote=F, col.names=NA)

resMF = results(ddsMF, contrast=c("MF_condition",current,other))
resMF
summary(resMF)
resMF_Orderedlfc <- resMF[order(resMF$log2FoldChange),]
head(resMF_Orderedlfc)

plot.new()
#identify(resMF$baseMean, resMF$log2FoldChange, labels = row.names(resMF))
#write.csv(as.data.frame(resMF), file= paste0(current,'_VS_',other,'.MF.csv'))

write.csv(as.data.frame(resMF_Orderedlfc), file= paste0(current,'_VS_',other,'.MF.tRNA.csv'))

# Volcano plot


pdf(file = paste0(current,'_VS_',other,'.volcano.tRNA.pdf'),   # The directory you want to save the file in
    width = 10, # The width of the plot in inches
    height = 10) # The height of the plot in inches
par(mar=c(5,5,5,5), cex=1.0, cex.main=1.4, cex.axis=1.4, cex.lab=1.4)
topT <- as.data.frame(resMF)
#Adjusted P values (FDR Q values)
with(topT, plot(log2FoldChange, -log10(padj), pch=20, main=paste0(current,' VS ',other,'volcano'), cex=1.0, xlab=bquote(~Log[2]~fold~change), ylab=bquote(~-log[10]~Q~value)))
with(subset(topT, padj<0.05 & abs(log2FoldChange)>1), points(log2FoldChange, -log10(padj), pch=20, col="red", cex=0.5))
#Add lines for absolute FC>2 and P-value cut-off at FDR Q<0.05
abline(v=0, col="black", lty=3, lwd=1.0)
abline(v=-1, col="black", lty=4, lwd=2.0)
abline(v=1, col="black", lty=4, lwd=2.0)
abline(h=-log10(max(topT$pvalue[topT$padj<0.05], na.rm=TRUE)), col="black", lty=4, lwd=2.0)
dev.off()





